In [182]:
from lark import Lark
import dis
import ast

In [183]:
grammar = r"""
?block: "{" (stmt ";") * "}"

stmt: "global" WORD expr -> assign_global
    | "local" WORD expr -> assign_local
    | "fn" WORD "(" WORD * ")" block -> declare_function
    | "return" expr -> return
    | "if" expr block ("elif" expr block)* ("else" block)? -> conditional
    | "while" expr block -> while
    | "print" expr -> print
    | expr
    




?expr: term "==" expr -> eq
    | term "<" expr -> lt
    | term ">" expr -> gt
    | term "<=" expr -> le
    | term ">=" expr -> ge
    | term "!=" expr -> ne
    | "!" expr -> neg
    | expr "+" term -> add
    | expr "-" term -> sub
    | term
    
?term: term "*" factor -> mul
    | term "/" factor -> sub
    | factor

?factor: NUMBER
    | WORD
    | ESCAPED_STRING
    | WORD "(" expr* ")" -> call
    | "True"
    | "False"
    | "None"
    | "(" expr ")"

COMMENT: /\(#[\s\S]*#\)/

%import common.NUMBER
%import common.ESCAPED_STRING
%import common.WORD
%import common.WS
%ignore WS
%ignore COMMENT

"""

In [184]:
parser = Lark(grammar, start='block', parser='lalr')

In [258]:
tree = parser.parse("""
{
 print 2+2*9;
}
""")

In [259]:
print tree.pretty()

print
  add
    2
    mul
      2
      9



In [260]:
tree

Tree(print, [Tree(add, [Token(NUMBER, '2'), Tree(mul, [Token(NUMBER, '2'), Token(NUMBER, '9')])])])

In [261]:
class Code(object):
    
    def __init__(self):
        self.code = []
        self.names = []
        self.consts = []
        self.globals = {}

In [262]:
co = Code()

def visit(node):
    if hasattr(node, 'children'):

        for child in node.children:
            visit(child)
        if node.data in ('add', 'sub', 'mul', 'div'):
            co.code.append((node.data.upper(), None))
        elif node.data == 'print':
            co.code.append(('PRINT', None))
        elif node.data == 'stmt':
            co.code.append(('POP', None))
        elif node.data == 'block':
            pass
        else:
            raise Exception('unknown non-leaf node', node.data)
            
    else:
        
        if node.type == 'NUMBER':
            co.code.append(('PUSH', int(node.value)))
#         elif node.type == 'WORD':
#             co.
        else:
            raise Exception('unknown leaf node', node.type)


In [263]:
visit(tree)
co.code

[('PUSH', 2),
 ('PUSH', 2),
 ('PUSH', 9),
 ('MUL', None),
 ('ADD', None),
 ('PRINT', None)]

In [264]:
stack = []

def vm(code):
    for op, arg in code:
        if op == 'PUSH':
            stack.append(arg)
        elif op in ('ADD', 'SUB', 'MUL', 'DIV'):
            b = stack.pop()
            a = stack.pop()
            if op == 'ADD':
                stack.append(a+b)
            elif op == 'SUB':
                stack.append(a-b)
            elif op == 'MUL':
                stack.append(a*b)
            else:
                stack.append(a/b)
        elif op == 'PRINT':
            print stack.pop()
            
        elif op == 'POP':
            stack.pop()
            
        else:
            raise Exception('unknown operation', op)
            
vm(co.code)
stack

20


[]

In [225]:
co = compile("""

x=9
z=13
z=9

print x+z

11

""", 'temp', 'exec')

In [226]:
dis.dis(co)

  3           0 LOAD_CONST               0 (9)
              3 STORE_NAME               0 (x)

  4           6 LOAD_CONST               1 (13)
              9 STORE_NAME               1 (z)

  5          12 LOAD_CONST               0 (9)
             15 STORE_NAME               1 (z)

  7          18 LOAD_NAME                0 (x)
             21 LOAD_NAME                1 (z)
             24 BINARY_ADD          
             25 PRINT_ITEM          
             26 PRINT_NEWLINE       

  9          27 LOAD_CONST               2 (None)
             30 RETURN_VALUE        


In [208]:
co.co_names

('x', 'z')

In [214]:
co.co_consts

(9, 13, None)

In [227]:
co.co_code

'd\x00\x00Z\x00\x00d\x01\x00Z\x01\x00d\x00\x00Z\x01\x00e\x00\x00e\x01\x00\x17GHd\x02\x00S'